In [1]:
import sys
import math as np


1\. Write a function that converts number representation (bin<->dec<->hex)

In [2]:
def convert(x,input_base,output_base):
    n=int(str(x),input_base)
    conv_num=[]
    i=0
    while not n==0:
        R = n%output_base
        if(R>=input_base): R = chr(ord('A') + R-input_base)
        conv_num.insert(0,R)
        n = n//output_base
        i+=1
    return conv_num
print(convert(24,10,16))

[1, 8]


2\. Write a function that converts a 32 bit word into a single precision floating point (i.e. interprets the various bits as sign, mantissa and exponent)

In [3]:
def spfp(bits):
    bits = list(bits)
    bits.reverse()
    f_bit = bits[0:23]  #first 22 bits
    f_bit = [int(i) for i in f_bit] #convert to numbers 
    e_bit = bits[23:31]
    e_bit = [int(i) for i in e_bit] #convert to numbers 
    s_bit = int(bits[31]) #sign bit
    
    exponent=sum([e_bit[i]*(2**i) for i in range(len(e_bit))])

    mantissa=1
    for i in range(len(f_bit)):
        mantissa+=f_bit[len(f_bit)-(i+1)]*(2**(-i-1))
    num = ((-1)**s_bit)*mantissa*(2**(exponent - 127))
    
    return num

print(spfp("00111111100011001100110011001101"))

1.100000023841858


3\. Write a program to determine the underflow and overflow limits (within a factor of 2) for python on your computer. 

**Tips**: define two variables inizialized to 1 and halve/double them enough time to exceed the under/over-flow limits  

In [4]:

x=1.
i=0
while x/2>0:
    x=x/2
    i+=1
print('min value available ='+str(x)+' with '+str(i)+' iterations')

x=1.
i=0
#while x*2<sys.maxsize:
while x*2<np.inf:
    x=x*2
    i+=1
print('max value available ='+str(x)+' with '+str(i)+' iterations')


min value available =5e-324 with 1074 iterations
max value available =8.98846567431158e+307 with 1023 iterations


4\. Write a program to determine the machine precision

**Tips**: define a new variable by adding a smaller and smaller value (proceeding similarly to prob. 2) to an original variable and check the point where the two are the same 

In [5]:
var1=2
var2=3
i=-1
while(var2!=var1):
    var2=var1+10**i
    i=i-1
    
print(i)
print(var2)

-17
2.0


5\. Write a function that takes in input three parameters $a$, $b$ and $c$ and prints out the two solutions to the quadratic equation $ax^2+bx+c=0$ using the standard formula:
$$
x=\frac{-b\pm\sqrt{b^2-4ac}}{2a}
$$

(a) use the program to compute the solution for $a=0.001$, $b=1000$ and $c=0.001$

(b) re-express the standard solution formula by multiplying top and bottom by $-b\mp\sqrt{b^2-4ac}$ and again find the solution for $a=0.001$, $b=1000$ and $c=0.001$. How does it compare with what previously obtained? Why?

(c) write a function that compute the roots of a quadratic equation accurately in all cases

In [6]:
#def realOrComplex(function):
#    def wrapper(arg):
#        func = finction(arg)
#        if len(func)==2:
#            print('Solutions are real')
#        else:
#            print('Solution are complex')
#        return func
#    return wrapper


def zerosParabola(a,b,c): #a,b,c coefficient of equation ax^2+bx+c=0
    delta=b**2-4*a*c
    #x1=(-b+mp.sqrt(delta))/2/a
    #x2=(-b-mp.sqrt(delta))/2/a
    x1=(-b+delta**(1/2))/2/a
    x2=(-b-delta**(1/2))/2/a
    return x1, x2
    
def zerosAlt(a,b,c):
    delta=b**2-4*a*c
    x1=(-b+delta**(1/2))*(-b-delta**1/2)/2/a/(-b-delta**1/2)
    x2=(-b-delta**(1/2))*(-b+delta**1/2)/2/a/(-b+delta**1/2)
    return x1, x2

a=0.001
b=1000
c=0.001
#@realOrComplex
print(zerosParabola(a,b,c))
print(zerosAlt(a,b,c))
print('The two solutions are slightly different just in the first solution\'s least significative digits')

def zerosParCorrect(a,b,c):
    delta=b**2-4*a*c
    x1=-b/(2*a)+np.sqrt(delta)/(2*a)
    x2=-b/(2*a)-np.sqrt(delta)/(2*a)
    return x1, x2

print(zerosParCorrect(a,b,c))


(-9.999894245993346e-07, -999999.999999)
(-9.999894245993344e-07, -999999.999999)
The two solutions are slightly different just in the first solution's least significative digits
(-1.00000761449337e-06, -999999.999999)


6\. Write a program that implements the function $f(x)=x(x−1)$

(a) Calculate the derivative of the function at the point $x = 1$ using the derivative definition:

$$
\frac{{\rm d}f}{{\rm d}x} = \lim_{\delta\to0} \frac{f(x+\delta)-f(x)}{\delta}
$$

with $\delta = 10^{−2}$. Calculate the true value of the same derivative analytically and compare with the answer your program gives. The two will not agree perfectly. Why not?

(b) Repeat the calculation for $\delta = 10^{−4}, 10^{−6}, 10^{−8}, 10^{−10}, 10^{−12}$ and $10^{−14}$. How does the accuracy scales with $\delta$?

In [7]:
def f(x):
    return x*(x-1)

def derivate(function,x,delta):
    deriv=(function(x+delta)-function(x))/delta
    return deriv

delta=10**-2
print(derivate(f,1,delta))

print('Now lets try with different values of delta')

for i in range(1,8):
    delta=10**(-2*i)
    print(derivate(f,1,delta))
    
print('It can be observed that the accuracy increases as delta decreases until the valuse delta= '+str(10**-10)+' after that the accuracy decreases')

1.010000000000001
Now lets try with different values of delta
1.010000000000001
1.0000999999998899
1.0000009999177333
1.0000000039225287
1.000000082840371
1.0000889005833413
0.9992007221626509
It can be observed that the accuracy increases as delta decreases until the valuse delta= 1e-10 after that the accuracy decreases


7\. Consider the integral of the semicircle of radius 1:
$$
I=\int_{-1}^{1} \sqrt(1-x^2) {\rm d}x
$$
which it's known to be $I=\frac{\pi}{2}=1.57079632679...$.
Alternatively we can use the Riemann definition of the integral:
$$
I=\lim_{N\to\infty} \sum_{k=1}^{N} h y_k 
$$

with $h=2/N$ the width of each of the $N$ slices the domain is divided into and where
$y_k$ is the value of the function at the $k-$th slice.

(a) Write a programe to compute the integral with $N=100$. How does the result compares to the true value?

(b) How much can $N$ be increased if the computation needs to be run in less than a second? What is the gain in running it for 1 minute? 


In [31]:
def RiemannInt(N,y):
    h=2/N
    return sum(h*y[k] for k in range(N))


def f(N):
    x=[-1+2/(N-1)*i for i in range(N)]
    return [np.sqrt(1-x[i]**2) for i in range(N)]

N=100


y_k=f(N)
print('N= '+str(len(y_k))+':')
print(RiemannInt(N,f(N)))
print('The result just obtained is just approximately correct but not strictly equal to the real value since the number of iterations is too small')

import time

N = 100000
stop = 0
while stop==0:
    t1 = time.time() 
    RiemannInt(N,f(N))
    t2 = time.time() - t1
    if(t2>1):
        stop = 1
    N += 10000
Nmax = N-10000
print("Max Num of iteration in " + str(t2) +" seconds: about " + str(Nmax))
print('Then max number of iterations in 1 miunte is: '+str(Nmax*60))

temp=RiemannInt(Nmax,f(Nmax))
print('N= '+str(Nmax)+':')
print(str(temp))
#running code for 1 minute

#Nmax_minute = Nmax*60

t1 = time.time()
res = RiemannInt(Nmax*60,f(Nmax*60))   
t2 = time.time() - t1
print("The result after " +str(t2)+ " seconds of iterations is " + str(res))


N= 100:
1.5534179294048955
The result just obtained is just approximately correct but not strictly equal to the real value since the number of iterations is too small
Max Num of iteration in 1.0701367855072021 seconds: about 1440000
Then max number of iterations in 1 miunte is: 86400000
N= 1440000:
1.5707952350016519
The result after 59.52363038063049 seconds of iterations is 1.570796308612953
